# Inception + Resnet 모델

## 하이퍼 파라미터 : 1. 모듈

In [1]:
import os
import sys
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import time
import psutil
import random
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

## 하이퍼 파라미터 : 2. 데이터셋

In [2]:
ROOT_DATA_FOLDER = '../../../dataset/train_sample_videos'
REAL_DATA_PATH = os.path.join(ROOT_DATA_FOLDER, 'real')
FAKE_DATA_PATH = os.path.join(ROOT_DATA_FOLDER, 'fake')
REAL_DATASET_LIST = os.listdir(REAL_DATA_PATH)
FAKE_DATASET_LIST = os.listdir(FAKE_DATA_PATH)

print(f"REAL 영상 수 : {len(REAL_DATASET_LIST)}, 경로 : {REAL_DATA_PATH}")
print(f"FAKE 영상 수 : {len(FAKE_DATASET_LIST)}, 경로 : {FAKE_DATA_PATH}")

REAL 영상 수 : 46858, 경로 : ../../../dataset/train_sample_videos/real
FAKE 영상 수 : 265658, 경로 : ../../../dataset/train_sample_videos/fake


## 유틸리티 정의 : 1. 데이터 전처리 클래스
 - REAL 데이터셋 리스트와 FAKE 데이터셋 라벨링 후 합치기
 - 데이터셋이 크므로 Generator를 사용해 변수 할당을 줄여 메모리를 효율적으로 사용한다

In [ ]:
# 라벨링을 해주는 Generator
class Deepfake_Preprocessor:
    """
    author : RHIE MINHYUNG
    describe: 라벨링된 딥페이크 이미지셋 부여 0-fake 1-real
    [usage]
      #> datasets = Deepfake_Preprocessor()
      #> = datasets.load_data()
    """
    
    def __init__(self):
        
        self.cnt_real = 0
        self.cnt_fake = 0
        
        print("[Deepfake_Preprocessor 생성을 시작합니다...]")
        self.real_data_len = len(REAL_DATASET_LIST)
        self.fake_data_len = len(FAKE_DATASET_LIST)
        #--이미지 데이터 불러오고 list로 바꿔주기
        real_data = [self.show_process(REAL_DATA_PATH, img, data_type="real") for img in REAL_DATASET_LIST]
        print("\n......REAL 데이터 완료")
        fake_data = [self.show_process(FAKE_DATA_PATH, img, data_type="fake") for img in FAKE_DATASET_LIST]
        print("\n......FAKE 데이터 완료")
        print("[진행중1/5]이미지 데이터 불러오고 list로 바꿔주기")
        
        #--Fake=0, REAL=1 라벨을 부여해주는 제너레이터 생성
        labeled_real_gen = self.real_data_labelling(real_data)
        labeled_fake_gen = self.fake_data_labelling(fake_data)
        print("[진행중2/5]Fake=0, REAL=1 라벨을 부여해주는 제너레이터 생성")
    
        #--제너레이터 실행
        labeled_real_data = list(labeled_real_gen)
        labeled_fake_data = list(labeled_fake_gen)
        print("[진행중3/5]제너레이터 실행")
        
        #--라벨링된 REAL과 FAKE를 합친후 랜덤으로 섞기
        labeled_data = labeled_real_data + labeled_fake_data
        random.shuffle(labeled_data)
        print("[진행중4/5]라벨링된 REAL과 FAKE를 합친후 랜덤으로 섞기")
        
        #--numpy로 casting
        self.datasets = np.array(labeled_data)
        print("[완료5/5] datasets 구성: ", self.datasets.shape)
        
    def load_data(self):
        #Normalization
        X = self.datasets
        Y = to_categorical(Y, 2)

        #Reshape
        X = X.reshape(-1, 128, 128, 3)

        #Train-Test split
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
    
    def real_data_labelling(self, real_data):
        for img_array in real_data:
            labeled_real_set = (img_array, 1)
            yield labeled_real_set

    def fake_data_labelling(self, fake_data):
        for img_array in fake_data:
            labeled_fake_set = (img_array, 0)
            yield labeled_fake_set
    
    def show_process(self, path, filename, data_type=None):
        if(data_type == "real"):
            sys.stdout.write("\r"+"["+str(self.cnt_real)+" / "+str(self.real_data_len)+"]")
            self.cnt_real += 1
        elif(data_type == "fake"):
            sys.stdout.write("\r"+"["+str(self.cnt_fake)+" / "+str(self.fake_data_len)+"]")
            self.cnt_fake += 1
        return img_to_array(load_img(os.path.join(path, filename))) / 255.0

## 데이터셋 준비 ( Preprocess ) 
 - REAL 데이터셋 리스트와 FAKE 데이터셋 라벨링 후 합치기
 - 데이터셋이 크므로 Generator를 사용해 변수 할당을 줄여메모리를 효율적으로 사용한다

In [4]:
# 메모리 사용을 체크해주는 객체 process 생성
process = psutil.Process(os.getpid())

# 라벨링 실행전 메모리 상태 체크
before_time = time.clock()
mem_before = process.memory_info().rss / 1024 / 1024


datasets = Deepfake_Preprocessor()

# 라벨링 실행후 메모리 상태 체크
after_time = time.clock()
mem_after = process.memory_info().rss / 1024 / 1024
total_time = after_time - before_time

print('시작 전 메모리 사용량: {} MB'.format(mem_before))
print('종료 후 메모리 사용량: {} MB'.format(mem_after))
print('총 소요된 시간: {:.6f} 초'.format(total_time))

NameError: name 'Deepfake_Preprocessor' is not defined

## 데이터 전처리
 - 메모리가 부족해 이미지 전부를 로딩할 수 없음

In [3]:
cnt = 1
def show_process(path, filename, data_type=None):
    global cnt
    if(data_type == "real"):
        sys.stdout.write("\r"+"["+str(cnt)+" / "+str(len(REAL_DATASET_LIST))+"]")
        cnt += 1
    elif(data_type == "fake"):
        sys.stdout.write("\r"+"["+str(self.cnt_fake)+" / "+str(self.fake_data_len)+"]")
        self.cnt_fake += 1
    return img_to_array(load_img(os.path.join(path, filename))) / 255.0
#     return (img_to_array(load_img(os.path.join(path, filename))) / 255.0).tolist()

cnt2 = 1
def show_process2(path, filename, data_type=None):
    global cnt2
    if(data_type == "real"):
        sys.stdout.write("\r"+"["+str(cnt2)+" / "+str(len(REAL_DATASET_LIST))+"]")
        cnt2 += 1
    elif(data_type == "fake"):
        sys.stdout.write("\r"+"["+str(self.cnt_fake)+" / "+str(self.fake_data_len)+"]")
        self.cnt_fake += 1
    return img_to_array(load_img(os.path.join(path, filename))) / 255.0     
#     return (img_to_array(load_img(os.path.join(path, filename))) / 255.0).tolist()

# 메모리 사용을 체크해주는 객체 process 생성
process = psutil.Process(os.getpid())
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    print('시작 전 메모리 사용량: {} MB'.format(process.memory_info().rss / 1024 / 1024))
    arrImg = [show_process(REAL_DATA_PATH, REAL_DATASET_LIST[i], data_type="real") for i in range(10)]
    print("")
    print("완료")
#     onlyImg = [show_process2(REAL_DATA_PATH, REAL_DATASET_LIST[i], data_type="real") for i in range(int(len(REAL_DATASET_LIST)/10))]
    print("")
    print("완료")
    print('시작 후 메모리 사용량: {} MB'.format(process.memory_info().rss / 1024 / 1024))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6')
시작 전 메모리 사용량: 1273.56640625 MB
[10 / 46858]
완료

완료
시작 후 메모리 사용량: 1276.4921875 MB


In [25]:
def real_data_labelling(real_data):
    for img_array in real_data:
        yield img_array
X = np.array(arrImg)
aagen = real_data_labelling(X)
Y = np.array(list(aagen))
Y

array([[[[0.5058824 , 0.4862745 , 0.4745098 ],
         [0.5058824 , 0.49411765, 0.47843137],
         [0.5058824 , 0.49803922, 0.48235294],
         ...,
         [0.07450981, 0.06666667, 0.05098039],
         [0.07450981, 0.06666667, 0.05098039],
         [0.07450981, 0.06666667, 0.05098039]],

        [[0.5058824 , 0.4862745 , 0.4745098 ],
         [0.5019608 , 0.49019608, 0.4745098 ],
         [0.5019608 , 0.49411765, 0.47843137],
         ...,
         [0.07843138, 0.07058824, 0.05490196],
         [0.07843138, 0.06666667, 0.05490196],
         [0.07843138, 0.06666667, 0.05490196]],

        [[0.5019608 , 0.48235294, 0.47058824],
         [0.49803922, 0.4862745 , 0.47058824],
         [0.5019608 , 0.49411765, 0.47843137],
         ...,
         [0.09019608, 0.07450981, 0.0627451 ],
         [0.09019608, 0.07450981, 0.0627451 ],
         [0.09019608, 0.07450981, 0.0627451 ]],

        ...,

        [[0.4392157 , 0.43137255, 0.42352942],
         [0.4392157 , 0.43137255, 0.41960785]

In [26]:
Y.shape

(10, 128, 128, 3)

In [ ]:
tf.keras.preprocessing.image.ImageDataGenerator(
    REAL_DATA_PATH,
    rescale=1./255
)

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
Inception_Resnet_model = keras.models.load_model('Inception_Resnet_Model.h5')

In [ ]:
Inception_Resnet_model.summary()